# Load required packages

In [ ]:
from datetime import datetime, timedelta, date

import ee
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer 
from sklearn.metrics import matthews_corrcoef, roc_auc_score, plot_confusion_matrix

# Load Land Classification Data 

In [ ]:
df = pd.read_csv('landclass_locust.csv')
df.head(5)

The `date` column needs to be parsed as a date object:

In [ ]:
df['date'] = pd.DatetimeIndex(df['date'])
df.head(5)

Count of Land Cover Classification (LCC) values:

In [ ]:
df['lcc'].value_counts()

Plot data in Kenya map


In [ ]:
kenya = gpd.read_file('kenya_shapefile/County.shp')
gdf = gpd.GeoDataFrame(
    df,
    crs=4326,
    geometry=[Point(xy) for xy in zip(df.long, df.lat)])


fig, ax = plt.subplots(1, 1,figsize=(6, 10))

kenya.plot(ax=ax, facecolor="none", edgecolor='black', lw=0.7)#color = 'yellow')
gdf.plot(column = 'lcc', ax = ax, legend=True)

# Extract Sentinel2 Band Values to Points

In [ ]:
The next several cells are *optional*. If you have a [Google Earth Engine (GEE) account](https://signup.earthengine.google.com/), you can run the code to login to GEE, download the Sentinel2 data, and convert it into a data frame. In the interest of time, we'll use cached data from a CSV file.

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

Create two date columns to define the date period of the Sentinel2 images

In [ ]:
df['date_min']= df['date'] -  pd.to_timedelta(16, unit='d')
df['date_max']= df['date'] +  pd.to_timedelta(16, unit='d')
df.head()

In [ ]:
def extract_to_point(x):
    bands_s2 = ['B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9']
    lat = x['lat']
    long = x['long']
    p = ee.Geometry.Point([long, lat])
    
    #select image based on following criteria:
    #* within the date range we defined
    #* sort all images by cloud cover 
    #* select images containing our point of image
    #* least cloud-covered images
    
    img = ee.ImageCollection('COPERNICUS/S2_SR') \
            .filterDate(x['date_min'].strftime('%Y-%m-%d'), x['date_max'].strftime('%Y-%m-%d')) \
            .sort('CLOUD_COVER') \
            .filterBounds(p) \
            .first()
    try: 
        # extract value from the above image to the point of interest
        temp = img.select(bands_s2).sampleRegions(collection=p, scale=10, geometries=True)
        info = temp.getInfo()
        print("info: ", info)
        print(" ")
        values = info['features'][0]['properties']
        print('values: ', values)
        print(' ')
    except:
        values = {'B11':0, 'B12':0, 'B2':0, 'B3':0, 'B4':0, 'B5':0, 'B6':0, 'B7':0, 'B8':0, 'B8A':0, 'B9':0}
    return values

DO NOT Run this command for the interest of time

In [ ]:
### result = df.apply(extract_to_point,axis=1,result_type='reduce')
### s2=pd.DataFrame(result.tolist())
### s2['lat']=df['lat']
### s2['long']=df['long']
### s2['date']=df['date']
### s2['lcc']=df['lcc']

An example showing what extract_to_point function is doing

In [ ]:
example = df.iloc[3:5, :].apply(extract_to_point, axis=1, result_type='reduce')
example

**Back to mandatory code**

For the interest of time, use cached data for the s2 dataset.

In [ ]:
s2 = pd.read_csv('s2.csv')
display(s2.head(5))
s2['NDVI'] = (s2['B8'] - s2['B4']) / (s2['B8'] + s2['B4'])
s2.drop(['lat', 'long', 'date'], axis=1, inplace=True)
lcc_code = {
  'Cultivated and managed vegetation/agriculture (cropland)': 0, 
  'Urban / built up': 1, 
  'Open forest, unknown': 2
}

s2['lcc'] = s2['lcc'].map(lcc_code)

display(s2.head(5))

# XGBoost Model 

Split X_train, X_test, y_train, y_test

In [ ]:
X = s2.drop(['lcc'], axis=1)
y = s2['lcc']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

Define searching grids and other parameters in GridSearchCV

In [ ]:
estimator = XGBClassifier(
    objective="multi:softprob",
    tree_method="hist",
    seed=42,
    nthread=-1,
)

kfold = RepeatedStratifiedKFold(n_splits=3, n_repeats=2, random_state=1234)

parameters = {
    #'eta' : [0.025, 0.5, 0.1],
    'max_depth': [6, 8, 10],
    'n_estimators': range(80, 100, 10),   
    'learning_rate': [0.1, 0.01, 0.05],
    #'min_child_weight': [1, 3, 5],
    #'gamma': [0, 0.5, 1],
    #'subsample': np.arange(0.6, 0.9, 0.1),
    #'colsample_bytree':[0.5, 0.75, 1.0],
    #'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}

fit_params = {
    'early_stopping_rounds': 10,
    'eval_set': [[X_test, y_test]]
}

mcc_scorer = make_scorer(matthews_corrcoef)

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring=mcc_scorer,
    cv=kfold,
    verbose=True
)

Fit the model

In [ ]:
grid_search.fit(X_train, y_train, **fit_params)

Let's see the best parameters found 

In [ ]:
grid_search.best_params_

Let's see the accuracy 

In [ ]:
accuracy_score(y_test, grid_search.predict(X_test))

Let's see the confusion matrix

In [ ]:
disp = plot_confusion_matrix(
    grid_search, X_test, y_test, 
    cmap=plt.cm.Blues,
    display_labels=['cropland', 'Urban', 'forest'])
disp.ax_.set_title("XGBoost Confusion Matrix")
plt.show()

Feature importance

In [ ]:
grid_search.best_estimator_.feature_importances_

plt.bar(x = X_train.columns, height = grid_search.best_estimator_.feature_importances_)
plt.title("Feature Importance")
plt.show()


## Bonus: how to extract time-series Sentinel2 data to points

First get a small subset of the entire dataset to save time

In [ ]:
df.sort_values(by=['date'], inplace=True)
subset = df.iloc[:5, :3]
subset

In [ ]:
bands_s2 = ["B11", "B12", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9"]

def extract_timeseries_to_point(x):
    lat = x["lat"]
    long = x["long"]
    p = ee.Geometry.Point([long,lat])
    
    # Create an image collection with 6-month's sentinel2 images, one for each month
    d = x["date"]
    date_list = [(d + timedelta(30*i)).strftime('%Y-%m-%d') for i in range(-3, 4)]
    image_ls = []
    for i in range(0, 6):
        s =  ee.ImageCollection("COPERNICUS/S2_SR")\
               .filterDate(date_list[i], date_list[i+1]) \
               .sort("CLOUD_COVER") \
               .filterBounds(p) \
               .first()
        image_ls.append(s)
    img_col = ee.ImageCollection(image_ls)
    
    # Extract values from the image collection to the point of interest and save to a csv
    def extract(img):
        return img.select(bands_s2).sampleRegions(collection=p, scale=10, geometries=True)
    
    newft = ee.FeatureCollection(img_col.map(extract)).flatten()
    f = newft.getInfo()["features"]
    keys = f[0]["properties"].keys()
    values = zip(
      f[0]["properties"].values(),
      f[1]["properties"].values(), 
      f[2]["properties"].values(),
      f[3]["properties"].values(),
      f[4]["properties"].values(),
      f[5]["properties"].values())
    dictionary = dict(zip(keys, values))
    
    display(pd.DataFrame.from_dict(dictionary))
    pd.DataFrame.from_dict(dictionary).to_csv(f"timeseries/df_{long}_{lat}_{d.strftime('%Y-%m-%d')}.csv")
    

In [ ]:
subset.apply(extract_timeseries_to_point, axis=1)